In [ ]:
!pip install selenium
!pip install html5lib
!pip install python-csv

In [2]:
class Anime:
    def __init__(self):
        self._name = ''
        self._release = ''
        self._imdb = ''
        self._votes = ''
        self._detail = ''
        self._image = ''
    
    def getAnime(self, temp):
        temp.append(self._name)
        temp.append(self._release)
        temp.append(self._imdb)
        temp.append(self._votes)
        temp.append(self._detail)
        temp.append(self._image)
        

    # def setData(self, name, release, imdb, votes, detail, image):
    #     self._name = name
    #     self._release= release
    #     self._imdb = imdb
    #     self._votes = votes
    #     self._detail = detail
    #     self._image = image


In [1]:
# Thêm thư viện cho project
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
from config_repository import ConfigRepository
import pandas as pd


In [7]:

def getPageFromUrl(url):
    driver = webdriver.Chrome(executable_path='chromedriver.exe')
    driver.get(url)
    driver.maximize_window()
    sleep(3)
    scroll = 800
    page_source = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()
    return page_source

def getDomain(url):
    domain = urlparse(url).netloc
    return domain

In [ ]:
# domain = urlparse('search/keyword/?keywords=anime').netloc

# if domain:
#     print(domain)
# else: print('no domain')


In [4]:
def getDataInPage(page_source):
    cfs = ConfigRepository()
    pattern = cfs.getPatternByUrl(url)
    list_records = page_source.select(pattern["list"])
    anime = Anime()
    records = []
    domain = getDomain(url)
    for record in list_records:
        anime._name = record.select_one(pattern["title"]).text
        # print(anime._name)
        subdomain = record.select_one(pattern["title"])["href"]
        if urlparse(subdomain).netloc:
            anime._detail = subdomain
        else:
            anime._detail = domain + subdomain
        # print(anime._detail)
        anime._imdb = record.select_one(pattern["imdb"]).text.split()
    # print(anime._imdb)
        votes = record.select_one(pattern["votes"]).text.split()
        if len(votes) > 1:
            anime._votes = votes[pattern["index-votes"]]
        else:
            anime._votes = votes
    # print(anime._votes)
        release = record.select_one(pattern["release"]).text.split()
    # print(release)
        if len(release) > 2:
            anime._release = (
                release[pattern["index-release-start"]]
                + " - "
                + release[pattern["index-release-end"]]
            )
        else:
            anime._release = record.select_one(pattern["release"]).text
    # print(anime._release)
        anime._image = record.select_one(pattern["thumbnail"])[pattern["get-thumbnail"]]
        temp = []
        anime.getAnime(temp)
        records.append((temp),)
    # print(records)
    return records




In [13]:
url = "https://www.imdb.com/search/keyword/?keywords=anime"

page_source = getPageFromUrl(url)
data = getDataInPage(page_source)
url = "https://myanimelist.net/topanime.php"
page_source = getPageFromUrl(url)
data = data + getDataInPage(page_source)
# for tmp in data:
#     print(tmp, '----')



['Attack on Titan', '(2013–2022)', ['9.0'], ['256,360'], 'www.imdb.com/title/tt2560140/?ref_=kw_li_tt', 'https://m.media-amazon.com/images/M/MV5BMTY5ODk1NzUyMl5BMl5BanBnXkFtZTgwMjUyNzEyMTE@._V1_UX140_CR0,0,140,209_AL_.jpg'] ----
['Demon Slayer: Kimetsu no Yaiba', '(2019– )', ['8.7'], ['54,388'], 'www.imdb.com/title/tt9335498/?ref_=kw_li_tt', 'https://m.media-amazon.com/images/M/MV5BZjZjNzI5MDctY2Y4YS00NmM4LTljMmItZTFkOTExNGI3ODRhXkEyXkFqcGdeQXVyNjc3MjQzNTI@._V1_UY209_CR5,0,140,209_AL_.jpg'] ----
['One Piece', '(1999– )', ['8.8'], ['87,068'], 'www.imdb.com/title/tt0388629/?ref_=kw_li_tt', 'https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png'] ----
['My Hero Academia', '(2016– )', ['8.4'], ['48,514'], 'www.imdb.com/title/tt5626028/?ref_=kw_li_tt', 'https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png'] ----
['Cowboy Bebop', '(1998–1999)', ['8.9'], ['98,971'], 'www.imdb.com/title/tt0213338/?ref_=kw_li_tt', 'https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png'] --